<a href="https://colab.research.google.com/github/fatinirq/Spoiler-Review/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import spacy
import pandas as pd
from torchtext.legacy import data
from torchtext.legacy.data import TabularDataset
from sklearn.model_selection import train_test_split
from updatedRNN_model import RNN
import torch.nn as nn


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/SpoilerData/LDA_CData.csv')
#data = [json.loads(line) for line in open('/content/drive/MyDrive/SpoilerData/CData.json', 'r')]
df=pd.DataFrame(data)

In [ ]:
del data
#sample = df.sample(frac=0.05)
#print(sample)

In [ ]:
df['spoiled'].value_counts()

0    17102931
1      569724
Name: spoiled, dtype: int64

In [ ]:
pd.show_versions()

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)



INSTALLED VERSIONS
------------------
commit           : 66e3805b8cabe977f40c05259cc3fcf7ead5687d
python           : 3.7.13.final.0
python-bits      : 64
OS               : Linux
OS-release       : 5.4.188+
Version          : #1 SMP Sun Apr 24 10:03:06 PDT 2022
machine          : x86_64
processor        : x86_64
byteorder        : little
LC_ALL           : None
LANG             : en_US.UTF-8
LOCALE           : en_US.UTF-8

pandas           : 1.3.5
numpy            : 1.21.6
pytz             : 2022.1
dateutil         : 2.8.2
pip              : 21.1.3
setuptools       : 57.4.0
Cython           : 0.29.30
pytest           : 3.6.4
hypothesis       : None
sphinx           : 1.8.6
blosc            : None
feather          : 0.4.1
xlsxwriter       : None
lxml.etree       : 4.2.6
html5lib         : 1.0.1
pymysql          : None
psycopg2         : 2.7.6.1 (dt dec pq3 ext lo64)
jinja2           : 2.11.3
IPython          : 5.5.0
pandas_datareader: 0.9.0
bs4              : 4.6.3
bottleneck       : N

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

Applying code of this report [link text](https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24)

In [ ]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
           # result.append(lemmatize_stemming(token))
           result.append(token)
    return result

In [ ]:
df['index'] = df.index

In [ ]:
df.head()

,review_sentence,spoiled,review_id,index
0,"['special', 'book']",0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,0
1,"['started', 'slow', 'first', 'third', 'middle'...",0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,1
2,"['love', 'good', 'science', 'fiction', 'push',...",0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,2
3,"['hugo', 'winner', 'translated', 'original', '...",0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,3
4,"['instance', 'intermixing', 'chinese', 'revolu...",0,dfdbb7b0eb5a7e4c26d59a937e2e5feb,4


In [ ]:

doc_sample = df[df['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
["['emotion',", "'sexual',", "'tension',", "'heartbreak',", "'packed',", "'book',", "'many',", "'book',", "'twice',", "'size']"]


 tokenized and lemmatized document: 
['emotion', 'sexual', 'tension', 'heartbreak', 'packed', 'book', 'book', 'twice', 'size']


In [ ]:
processed_docs = df['review_sentence'].map(preprocess)
processed_docs[:10]

0                                      [special, book]
1    [started, slow, middle, started, interesting, ...
2    [love, good, science, fiction, push, thinking,...
3    [hugo, winner, translated, original, chinese, ...
4    [instance, intermixing, chinese, revolutionary...
5                               [book, science, alien]
6    [science, described, book, impressive, book, g...
7    [folding, proton, dimension, think, making, st...
8    [happen, seti, station, received, message, per...
9    [book, dark, like, think, human, reaction, dis...
Name: review_sentence, dtype: object

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
print(len(dictionary))

0 book
1 special
2 blew
3 interesting
4 middle
5 mind
6 slow
7 started
8 fiction
9 good
10 love
324907


In [ ]:
#del processed_docs df

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
print (len(dictionary))

57750


In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(0, 2),
 (368, 1),
 (1936, 1),
 (2260, 1),
 (2308, 1),
 (3394, 1),
 (4872, 1),
 (5006, 1)]

In [ ]:
bow_corpus[13004310]

[(27, 1), (46, 1), (1560, 1), (3001, 1), (4049, 1), (4589, 1), (8666, 1)]

In [ ]:

bow_corpus[4314]

[(621, 1), (1532, 1), (1888, 1), (2172, 1), (2456, 1), (2458, 1), (4382, 1)]

In [ ]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0],
                                               dictionary[bow_doc_4310[i][0]],
bow_doc_4310[i][1]))

Word 0 ("book") appears 2 time.
Word 368 ("tension") appears 1 time.
Word 1936 ("twice") appears 1 time.
Word 2260 ("emotion") appears 1 time.
Word 2308 ("sexual") appears 1 time.
Word 3394 ("packed") appears 1 time.
Word 4872 ("heartbreak") appears 1 time.
Word 5006 ("size") appears 1 time.


In [ ]:

lda_model = gensim.models.LdaMulticore(bow_corpus[:10000000], num_topics=10, id2word=dictionary, passes=2, workers=4)

In [ ]:
del df

In [ ]:
import gc
gc.collect()

0

In [ ]:
for index, score in sorted(lda_model[bow_corpus[1000]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 20)))


Score: 0.4450543522834778	 
Topic: 0.039*"want" + 0.013*"thing" + 0.010*"people" + 0.010*"understand" + 0.008*"thought" + 0.008*"human" + 0.008*"life" + 0.008*"okay" + 0.008*"world" + 0.007*"tell" + 0.007*"girl" + 0.006*"like" + 0.006*"think" + 0.006*"fuck" + 0.005*"head" + 0.005*"place" + 0.005*"said" + 0.004*"mind" + 0.004*"know" + 0.004*"heart"

Score: 0.35491031408309937	 
Topic: 0.039*"like" + 0.017*"read" + 0.014*"sure" + 0.012*"knew" + 0.011*"happens" + 0.010*"issue" + 0.010*"thing" + 0.010*"look" + 0.010*"happen" + 0.010*"facebook" + 0.009*"know" + 0.008*"later" + 0.008*"going" + 0.007*"weird" + 0.007*"work" + 0.007*"year" + 0.006*"spoiler" + 0.006*"finish" + 0.005*"crazy" + 0.005*"saying"

Score: 0.02501116320490837	 
Topic: 0.128*"book" + 0.049*"read" + 0.029*"series" + 0.023*"thing" + 0.021*"like" + 0.015*"love" + 0.015*"character" + 0.014*"reading" + 0.012*"think" + 0.012*"great" + 0.011*"favorite" + 0.011*"liked" + 0.011*"definitely" + 0.010*"feel" + 0.009*"time" + 0.009*

In [ ]:
print(type(lda_model))

<class 'gensim.models.ldamulticore.LdaMulticore'>


In [ ]:
print(lda_model)

LdaModel(num_terms=57750, num_topics=10, decay=0.5, chunksize=2000)


In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=60, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
unseen_document = df[0]['review_sentences	']
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))